In [ ]:
# main.py or analysis script

from scripts.macro_data import MacroDataManager
from scripts.bond_data import DataPaths, FXDataManager, BondDataManager
from scripts.market_data import MarketDataManager
from pathlib import Path

# --- Dependency functions assumed to be defined/imported ---
from utils import normalize_periods, calc_difference, calc_ratio, convert_to_usd, get_yf

# --- Set up paths ---
local_drive = Path("C:/Users/torin/OneDrive")
datamacro = Path("Desktop/Thesis/data/macro")

# --- Instantiate MacroDataManager ---
macro = MacroDataManager(
    local_drive=local_drive,
    datamacro=datamacro,
    normalize_func=normalize_periods,
    diff_func=calc_difference,
    ratio_func=calc_ratio,
    fx_converter=convert_to_usd,
    yf_loader=get_yf
)

# --- Generate macro variable DataFrames ---
gdp = macro.get_gdp(growth=False, lag=False)
gdp_growth = macro.get_gdp(growth=True, lag=False)[['Brazil_GDP', 'Italy_GDP']]
gdp_growth.columns = ['brazil_gdp_growth', 'italy_gdp_growth']

inflation = macro.get_inflation()
debt = macro.get_debt(gdp)
deficit = macro.get_deficit(gdp)
tradebal = macro.get_balance_of_trade(gdp)
exchange_rate = macro.get_fx()
foreign_reserves = macro.get_foreign_reserves(gdp)
non_residents = macro.get_debt_composition()

In [ ]:
paths = DataPaths(local_drive)
fx_manager = FXDataManager(paths)
bond_manager = BondDataManager(paths)

fx_data = fx_manager.get_combined_fx()
tenors = [2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 20, 25, 30]
monthly_spreads = bond_manager.load_sector_spreads(tenors)

In [ ]:
manager = MarketDataManager(get_yf, normalize_periods, Path('...'), Path('...'))
vix = manager.get_vix()
gold = manager.get_gold()

In [ ]:
########### merging final dataset ################

estimates = {}
for k,v in monhtly_sector_spreads.items():
    estimates[k] = pd.concat([gdp,
                              gdp_growth,
                              inflation, 
                              debt, 
                              deficit, 
                              exchange_rate,
                              foreign_reserves,
                              non_residents,
                              tradebal,
                              vix, 
                              ushighyield, 
                              gold, 
                              commods,
                              us3m,
                              contagion,
                              v], axis = 1).dropna()

def get_model_data(estimates, measure = 'spread'):

  estall = pd.concat(estimates).reset_index()
  del estall['level_0']
  estall['tenor'] = 'tenor_' + estall['tenor'].astype(str).apply(lambda x : str(x)[0:2].replace('.',''))
  
  if measure == 'spread':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['spread_BR_IT'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('spread_BR_IT_')
  
  elif measure == 'yield_br':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['yield BR'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('yield_br_')

  elif measure == 'yield_it':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['yield IT'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('yield_it_')
  
  estall = estall.set_index(['DATE']).drop([ 
      'tenor', 
      'yield BR', 
      'yield IT',
      'spread_BR_IT'], axis = 1).drop_duplicates().sort_index()

  model_data = pd.concat([tenorsdf, 
                          estall], axis = 1)

  model_data = model_data.dropna()

  cols_rename = {'DLSP - Dívida Fiscal Líquida':'br_divida_fiscal_liquida',
        'DLSP - dívida externa líquida - total': 'br_divida_externa_liquida',
        'DLSP - dívida interna líquida': 'br_divida_interna_liquida',
        'DLSP - Dívida Fiscal Líquida (% of pib)': 'br_divida_fiscal_liquida_pct_pib',
        'DLSP - dívida externa líquida - total (% of pib)': 'br_divida_externa_liquida_pib',
        'DLSP - dívida interna líquida (% of pib)': 'br_divida_interna_liquida_pib',
        'General Government: gross debt':'it_gross_debt',
        'trend - state deficit': 'it_trend_state_deficit',
        'NFSP - governo federal e Banco Central - juros nominais': 'br_juros_nominais',
        'NFSP - governo federal e Banco Central - nominal': 'br_deficit_nominal',
        'NFSP - governo federal e Banco Central - primário': 'br_deficit_primario',
        'United States_GDP':'us_gdp',
        'United States_GDPcapita':'us_gdp_per_capita',
        'Brazil_GDP':'br_gdp',
        'Italy_GDP':'it_gdp', 
        'Italy_GDPcapita':'it_gdp_per_capita',
          'Brazil_GDPcapita':'br_gdp_per_capita'}

  model_data.rename(cols_rename, axis =1, inplace=True)

  return model_data

get_model_data(estimates).to_csv(local_drive / databonds/ 'model_data_spread_br_it.csv')
get_model_data(estimates, measure = 'yield_it').to_csv(local_drive / databonds/ 'model_data_yield_it.csv')
get_model_data(estimates, measure = 'yield_br').to_csv(local_drive / databonds/ 'model_data_yield_br.csv')